# Minimización de Rastrigin en 3 dimensiones (tres variables)
## Definición de la función
Usando la función de Rastrigin definida como:

In [26]:
import numpy as np

K_A = 10

def rastrigin(x: np.ndarray, y: np.ndarray, z: np.ndarray) -> np.ndarray:
    return K_A * 3 \
           + np.power(x, 2) - K_A * np.cos(2*np.pi * x) \
           + np.power(y, 2) - K_A * np.cos(2*np.pi * y) \
           + np.power(z, 2) - K_A * np.cos(2*np.pi * z)

## Definición de cromosoma
Se compone unicamente por los números que representa, el dominio usual de la función de Rastrigin es de -5.12 a 5.12, por eso vamos a dividir el valor entero entre 100.

In [27]:
BITS = 10
DIVISION = 100
POWERS = np.array([2 ** i for i in range(BITS-2, -1, -1)])

class Cromosoma:
    """
    Se conforma por dos listas de bits, sirven como index lógico de POWERS
    """
    def __init__(self, *args):
        if len(args) == 0:
            self.x = np.array([np.random.choice([0, 1]) for i in range(0, BITS)])
            self.y = np.array([np.random.choice([0, 1]) for i in range(0, BITS)])
            self.z = np.array([np.random.choice([0, 1]) for i in range(0, BITS)])
        else:
            self.x = args[0]
            self.y = args[1]
            self.z = args[2]

    def __str__(self):
        return f'\n\tx:{self.x} \n\ty:{self.y} \n\tz:{self.z}'


    def to_int(self, index):
        if index == 0:
            signo = 1 if self.x[0] == 0 else -1
            index = self.x[1:] == 1
            return signo * np.sum(POWERS[index]) / DIVISION
        elif index == 1:
            signo = 1 if self.y[0] == 0 else -1
            index = self.y[1:] == 1
            return signo * np.sum(POWERS[index]) / DIVISION
        else:
            signo = 1 if self.z[0] == 0 else -1
            index = self.z[1:] == 1
            return signo * np.sum(POWERS[index]) / DIVISION

    @staticmethod
    def crossover(c1, c2):
        mitad = int(BITS / 2)
        x1 = np.append(c1.x[0:mitad], c2.x[mitad:], axis=None)
        x2 = np.append(c2.x[0:mitad], c1.x[mitad:], axis=None)
        y1 = np.append(c1.y[0:mitad], c2.y[mitad:], axis=None)
        y2 = np.append(c2.y[0:mitad], c1.y[mitad:], axis=None)
        z1 = np.append(c1.y[0:mitad], c2.y[mitad:], axis=None)
        z2 = np.append(c2.y[0:mitad], c1.y[mitad:], axis=None)
        return [Cromosoma(x1, y1, z1), Cromosoma(x2, y2, z2)]

    @staticmethod
    def mutar(c):
        index = np.random.randint(low=1, high=len(c.x))
        c.x[index] = 1 - c.x[index]
        index = np.random.randint(low=1, high=len(c.y))
        c.y[index] = 1 - c.y[index]
        index = np.random.randint(low=1, high=len(c.z))
        c.z[index] = 1 - c.z[index]
        return c


## Definición de presión selectiva

In [28]:
K_POBLACION = 10
K_BASE = 2

def presion_selectiva(poblacion: list[Cromosoma]) -> list[Cromosoma]:
    # Evaluación y búsqueda del mejor
    x = np.array([c.to_int(0) for c in poblacion])
    y = np.array([c.to_int(1) for c in poblacion])
    z = np.array([c.to_int(2) for c in poblacion])
    evaluacion = rastrigin(x, y, z)

    best = evaluacion.argmin()
    print("Best so far:")
    print("Value: " + str(poblacion[best]))
    print(f"Integer value: {x[best]}, {y[best]}, {z[best]}")
    print("Rastrigin function value: " + str(evaluacion[best]))

    # Cálculo de probabilidades
    indice_ordenado = evaluacion.argsort()
    ruleta = []
    potencia = K_POBLACION

    for i in indice_ordenado:
        probabilidad = K_BASE ** potencia
        ruleta.extend([i] * probabilidad)
        potencia -= 1

    # Nueva generación
    nueva = list[Cromosoma]()
    nueva.append(poblacion[indice_ordenado[0]])
    nueva.append(poblacion[indice_ordenado[1]])

    for i in range(2, int(K_POBLACION/2)):
        c1 = poblacion[np.random.choice(ruleta)]
        c2 = poblacion[np.random.choice(ruleta)]
        hijos = Cromosoma.crossover(c1, c2)

        hijos[0] = Cromosoma.mutar(hijos[0])
        hijos[1] = Cromosoma.mutar(hijos[1])

        nueva.extend(hijos)

    return nueva

## Ciclo de vida

In [29]:
poblacion = list[Cromosoma]()
nueva_poblacion = list[Cromosoma]()
generacion = 0

In [1]:
for _ in range(0, 100):
    if len(nueva_poblacion) == 0:
        poblacion = [Cromosoma() for _ in range(0, K_POBLACION)]
    else:
        poblacion = nueva_poblacion

    print('\nGeneración', generacion)
    nueva_poblacion = presion_selectiva(poblacion)
    generacion += 1

NameError: name 'nueva_poblacion' is not defined